<a href="https://colab.research.google.com/github/DinurakshanRavichandran/Visio-Glance/blob/Fundus-eye-disease-detection/Res_Mob_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#  MOUNT GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [3]:
#  DEFINE DATASET PATHS
DATA_DIR = '/content/drive/MyDrive/data set/dataset'
FEATURE_DIR = '/content/drive/MyDrive/extracted_feature'

#  FUNCTION TO GET IMAGE DIRECTORIES
def get_class_directories(base_directory):
    return {folder_name: os.path.join(base_directory, folder_name) for folder_name in os.listdir(base_directory) if os.path.isdir(os.path.join(base_directory, folder_name))}


    image_directories = get_class_directories(DATA_DIR)
class_names = list(image_directories.keys())
print(f"Classes found: {class_names}")


NameError: name 'image_directories' is not defined